In [19]:
import numpy as np                                     # Matlab like syntax for linear algebra and functions
import matplotlib.pyplot as plt                        # Plots and figures like you know them from Matlab
import seaborn as sns                                  # Make the plots nicer to look at
from iminuit import Minuit                             # The actual fitting tool, better than scipy's
import sys                                             # Modules to see files and folders in directories
from scipy import stats
r = np.random

In [2]:
sys.path.append('../AppStat2023/External_Functions')
from ExternalFunctions import UnbinnedLH, BinnedLH, Chi2Regression
from ExternalFunctions import nice_string_output, add_text_to_ax   

In [18]:
blind = True

In [3]:
def constant(x,c):
    return c

def fit(fit, x, y, sy):
    Minuit.print_level = 1    # Print result of fits (generally - can also be moved down to each fit instance)
    
    chi2fit = Chi2Regression(fit, x, y, sy)
    
    minuit_chi2 = Minuit(chi2fit, c=0)
    minuit_chi2.errordef = 1.0     # This is the definition for ChiSqaure fits
    minuit_chi2.migrad();          # This is where the minimisation is carried out! Put ";" at the end to void output
    return minuit_chi2

In [4]:
def weighted_mean(y_data, y_err):
    mean = fit(constant,  # fit function
               np.arange(len(y_data)), # arbitrary x_values
               y_data, # y-data
               y_err # y-error
              )
    for name in mean.parameters:
        value, error = mean.values[name], mean.errors[name]
        print(f"Fit value: {name} = {value:.5f} +/- {error:.5f}")
    w_mean = mean.values['c']
    w_err  = mean.errors['c'], 
    chi2_value = mean.fval
    Ndof_value = len(y_data)-1 # data-points - 1 parameter
    prob = stats.chi2.sf(chi2_value, Ndof_value)
    return w_mean, w_err, chi2_value, prob

In [67]:
# Vi er 'lige blinde' på alle målinger af samme gate, men ikke på hver gate... Skal de alle forskydes lige meget i stedet måske?
# Vi er ikke blinde på '0'

if blind:
    blinding = np.concatenate((np.zeros(1), r.normal(0, 5, size=5-1))) 
else:
    blinding = np.zeros(5)

#positioner af målere
person1_pos = np.array([19.25, 35.48, 53.45, 69.95, 87.75]) + blinding #cm
person1_pos += -person1_pos[0] # Sætter den første som 0 punkt
person1_pos_sigma = np.array([0.05]) #cm

person2_pos = np.array([17, 33.6, 51.4, 67.7, 85.6]) + blinding #cm
person2_pos += -person2_pos[0]
person2_pos_sigma = np.array([0.1]) #cm

person3_pos = np.array([12.10, 29.85, 46.35, 64.05, 80.65]) + blinding #cm
person3_pos += -person3_pos[0]
person3_pos_sigma = np.array([0.05]) #cm

person4_pos = np.array([11.24, 29.00, 45.43, 63.01, 79.84]) + blinding #cm
person4_pos += -person4_pos[0]
person4_pos_sigma = np.array([0.02]) #cm


# Alternative weighted mean:
#weighted_sigma = 1/np.sqrt(person1_pos_sigma**(-2) + person2_pos_sigma**(-2) + person3_pos_sigma**(-2) + person4_pos_sigma**(-2))
#weighted_pos = person1_pos*person1_pos_sigma**(-2) + person2_pos*person2_pos_sigma**(-2) + person3_pos*person3_pos_sigma**(-2) + person4_pos*person4_pos_sigma**(-2)
#weighted_pos *= weighted_sigma**(2)

gate_pos = np.ones(len(person1_pos))
gate_err = np.ones(len(person1_pos))

for i in range(len(person1_pos)):
    mean, err, chi2, prob = weighted_mean(
        [person1_pos[i], person2_pos[i], person3_pos[i], person4_pos[i]],
        [person1_pos_sigma[0], person2_pos_sigma[0], person3_pos_sigma[0], person4_pos_sigma[0]]
                                   )
    gate_pos[i]+=mean
    gate_err[i]+=err[0]

Fit value: c = 0.00000 +/- 0.01715
Fit value: c = 18.61141 +/- 0.01715
Fit value: c = 29.36098 +/- 0.01715
Fit value: c = 56.88532 +/- 0.01715
Fit value: c = 73.89762 +/- 0.01715


In [53]:
if blind:
    blinding = r.normal(0, 3, size=2)
else:
    blinding = np.zeros(2)

#trigonometri
person1_height_length = np.array([22.05, 89.9])+blinding #cm height, length
person1_hl_sigma = np.array([0.05]) #cm

person2_height_length = np.array([22.1, 90.1])+blinding #cm height, length
person2_hl_sigma = np.array([0.2]) #cm

person3_height_length = np.array([22.7, 93.1])+blinding #cm height, length
person3_hl_sigma = np.array([0.1]) #cm

person4_height_length = np.array([22.34, 89.19])+blinding #cm height, length
person4_hl_sigma = np.array([0.05]) #cm

height_length = np.zeros(len(person1_height_length))
hl_err = np.zeros(len(person1_height_length))

for i in range(len(person1_height_length)):
    mean, err, chi2, prob = weighted_mean(
        [person1_height_length[i], person2_height_length[i], person3_height_length[i], person4_height_length[i]],
        [person1_hl_sigma[0], person2_hl_sigma[0], person3_hl_sigma[0], person4_hl_sigma[0]]
                                   )
    height_length[i]+=mean
    hl_err[i]+=err[0]

Fit value: c = 19.89323 +/- 0.03288
Fit value: c = 90.89212 +/- 0.03288


In [113]:
if blind:
    blinding = np.ones(2)*r.normal(0,3, size=1) # Begge målinger forskydes med samme værdi
else:
    blinding = np.zeros(2)

# vinkel
person1_angle = np.array([13.1, 11.9]) + blinding #degrees - lille måleapperat, stort måleapperat
person1_angle_sigma = np.array([0.5]) #degrees

person2_angle = np.array([11, 13.5]) + blinding #degrees
person2_angle_sigma = np.array([0.5]) #degrees

person3_angle = np.array([14.5, 12.0]) + blinding #degrees
person3_angle_sigma = np.array([0.5]) #degrees

person4_angle = np.array([13.31, 13.34]) + blinding #degrees
person4_angle_sigma = np.array([0.05]) #degrees



angle = np.zeros(len(person1_angle))
angle_err = np.zeros(len(person1_angle))
for i in range(len(person1_height_length)):
    mean, err, chi2, prob = weighted_mean(
        [person1_angle[i], person2_angle[i], person3_angle[i], person4_angle[i]],
        [person1_angle_sigma[0], person2_angle_sigma[0], person3_angle_sigma[0], person4_angle_sigma[0]]
    )
    angle[i]+=mean
    angle_err[i]+=err[0]

Fit value: c = 13.94981 +/- 0.04927
Fit value: c = 13.96729 +/- 0.04927


In [124]:
if blind:
    blinding = np.ones(2)*r.normal(0,3, size=1)
else:
    blinding = np.zeros(2)
    
#vinkel efter vending - biased

person1_angle_turn = np.array([13.7, 14.2]) + blinding #degrees - lille måleapperat, stort måleapperat
person1_angle_turn_sigma = np.array([0.5]) #degrees


person2_angle_turn = np.array([14.0, 14.5]) + blinding #degrees
person2_angle_turn_sigma = np.array([0.5]) #degrees


person3_angle_turn = np.array([14.5, 14.0]) + blinding #degrees
person3_angle_turn_sigma = np.array([0.5]) #degrees

person4_angle_turn = np.array([12.9, 14.7]) + blinding #degrees
person4_angle_turn_sigma = np.array([0.05]) #degrees

angle_turn = np.zeros(len(person1_angle))
angle_turn_err = np.zeros(len(person1_angle))
for i in range(len(person1_height_length)):
    mean, err, chi2, prob = weighted_mean(
        [person1_angle_turn[i], person2_angle_turn[i], person3_angle_turn[i], person4_angle_turn[i]],
        [person1_angle_turn_sigma[0], person2_angle_turn_sigma[0], person3_angle_turn_sigma[0], person4_angle_turn_sigma[0]]
         )
    angle[i]+=mean
    angle_err[i]+=err[0]

Fit value: c = 14.41292 +/- 0.04927
Fit value: c = 16.16535 +/- 0.04927


In [164]:
if blind:
    blinding = r.normal(0,.05, size=1)
else:
    blinding = np.zeros(1)

#skinne afstand
person1_rail = np.array([0.6]) + blinding #cm
person1_rail_sigma = np.array([0.005]) #cm

person2_rail = np.array([0.6]) + blinding #cm
person2_rail_sigma = np.array([0.01]) #cm

person3_rail = np.array([0.65]) + blinding #cm
person3_rail_sigma = np.array([0.01]) #cm

person4_rail = np.array([0.57, 0.58, 0.59]) + blinding #cm
person4_rail_sigma = np.array([0.01]) #cm

rail = np.zeros(len(person1_rail))
rail_err = np.zeros(len(person1_rail))
for i in range(len(person1_rail)):
    mean, err, chi2, prob = weighted_mean(
        [person1_rail[i], person2_rail[i], person3_rail[i], person4_rail[i]],
        [person1_rail_sigma[0], person2_rail_sigma[0], person3_rail_sigma[0], person4_rail_sigma[0]]
                                   )
    rail[i]+=mean
    rail_err[i]+=err[0]


Fit value: c = 0.52218 +/- 0.00378


In [182]:
if blind:
    blinding = r.normal(0, 1)
else:
    blinding = np.zeros(1)

#Diameter kugle
person1_diam = np.array([14.91, 14.98, 14.97]) + blinding #mm
person1_diam_sigma = np.array([0.005]) #mm

person2_diam = np.array([15, 15.02, 15.02]) + blinding #mm
person2_diam_sigma = np.array([0.002]) #mm

person3_diam = np.array([14.96, 14.96, 14.98]) + blinding #mm
person3_diam_sigma = np.array([0.02]) #mm

person4_diam = np.array([14.92, 14.941, 14.878]) + blinding #mm
person4_diam_sigma = np.array([0.005]) #mm

diam = np.zeros(1)
diam_err = np.zeros(1)
for i in range(len(person1_rail)):
    mean, err, chi2, prob = weighted_mean(
        [*person1_diam, *person2_diam, *person3_diam, *person4_diam],
        [person1_diam_sigma[0], person1_diam_sigma[0], person1_diam_sigma[0], #Vi har 3 målinger med samme usikkerhed
         person2_diam_sigma[0], person2_diam_sigma[0], person2_diam_sigma[0],
         person3_diam_sigma[0], person3_diam_sigma[0], person3_diam_sigma[0],
         person4_diam_sigma[0], person4_diam_sigma[0], person4_diam_sigma[0]]
                                   )
    diam[i]+=mean
    diam_err[i]+=err[0]

Fit value: c = 14.51108 +/- 0.00100


In [17]:
# Værdier vi har
gate_pos, gate_err;          # Beliggenhed af gates +- sigma
height_length, hl_err;       # Højde & Længde af bane +- sigma på hver
angle, angle_err;            # Vinkel + bord +- sigma
angle_turn, angle_turn_err;  # Vinkel + bord efter vending +- sigma
rail, rail_err;              # Sliskebredde +- sigma
diam, diam_err;              # Bolddiameter +- sigma
